# CAI - Rebalancing data for November 2022 

## Setting

In [121]:
import pandas as pd
import numpy as np
import requests
import json
import time
import os
from decouple import config

In [122]:
from pycoingecko import CoinGeckoAPI
key = config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [123]:
import plotly.express as px
import plotly.graph_objects as go

In [124]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Data retrieving and filtering

### Methodology 

1. **Token inclusion criteria**
    1. Any token included in our index must meet all the criteria below:
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    1. The token must not have locking, minting or other patterns that would significantly disadvantage passive holders.
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe and Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [125]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [126]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

In [127]:
# wrapper avax
coins['avalanche-2']['addr'] = "0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7"

##### Market data

In [128]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)

In [129]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [130]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]

In [131]:
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
# tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]

In [132]:
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

### Token's supply check 

In [133]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > 0.3
sp_passed = tokens_market_data[sp_passed_index]

In [134]:
sp_passed

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
tether,Tether,USDT,6.853166e+10,3.0,1.001000,6.847311e+10,6.847311e+10,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7
binancecoin,BNB,BNB,4.506178e+10,4.0,276.090000,1.632770e+08,1.632770e+08,NaN
usd-coin,USD Coin,USDC,4.372039e+10,5.0,0.999666,4.383663e+10,4.375106e+10,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e
binance-usd,Binance USD,BUSD,2.163441e+10,7.0,1.001000,2.161689e+10,2.161689e+10,0x9c9e5fd8bbc25984b178fdce6117defa39d2db39
dai,Dai,DAI,5.769238e+09,15.0,0.999493,5.772164e+09,5.772164e+09,0xd586e7f844cea2f87f50152665bcbc2c279d8d70
...,...,...,...,...,...,...,...,...
ape-in,Ape In,APEIN,1.720782e+04,3653.0,0.458875,3.750000e+04,3.750000e+04,0x938fe3788222a74924e062120e7bfac829c719fb
nft-soccer-games,NFT Soccer Games,NFSG,1.500678e+04,3673.0,0.168948,2.000000e+05,8.923960e+04,0xf69c2fcd9128d49dfa22348c69177f9380438eb8
fortressdao,Fortress,FORT,1.472061e+04,3678.0,0.010500,1.409126e+06,1.401945e+06,0xf6d46849db378ae01d93732585bec2c4480d1fd5


### Liquidity check

In [135]:
api_urls = {
    'trader_joe': "https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange",
    'pangolin': "https://api.thegraph.com/subgraphs/name/pangolindex/exchange",
} 

In [136]:
tokens_ids = list(tokens_market_data['addr'].dropna())

In [137]:
gecko_ids = pd.Series(tokens_market_data.index, index=tokens_market_data['addr'].str.lower())
gecko_ids = gecko_ids.groupby(gecko_ids.index).first()

In [138]:
def trader_joe_liqs(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['trader_joe']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        id0 = gecko_ids[pair['token0']['id'].lower()]
        liqs[id0] = liqs.get(id0, 0) + liq
        
        id1 = gecko_ids[pair['token1']['id'].lower()]
        liqs[id1] = liqs.get(id1, 0) + liq 
        
    return liqs
    

In [139]:
def pangolin_liqs(tokens_ids):
    query = """
        query pairs($tokens_ids: [ID!]!) {
            pairs (first:1000, 
                where: {
                    token0_in: $tokens_ids, 
                    token1_in: $tokens_ids,
                    reserveUSD_gt: 10000
                }){
                token0 {
                    id
                    symbol
                }
                token1 {
                    id
                    symbol
                }
                reserveUSD
            }
        }
    """
    url = api_urls['pangolin']
    
    response = requests.post(url, json={'query': query, 'variables': {'tokens_ids': tokens_ids}})
    pairs = response.json()['data']['pairs']
    
    liqs = {}
    for pair in pairs:
        liq = float(pair['reserveUSD'])
        
        id0 = gecko_ids[pair['token0']['id'].lower()]
        liqs[id0] = liqs.get(id0, 0) + liq
        
        id1 = gecko_ids[pair['token1']['id'].lower()]
        liqs[id1] = liqs.get(id1, 0) + liq 
        
    return liqs
    

In [140]:
query_functions = {
    'trader_joe': trader_joe_liqs,
    'pangolin': pangolin_liqs
}

In [141]:
liqs_dict = {}
for dex, func in query_functions.items():
    liqs_dict[dex] = func(tokens_ids)

In [142]:
liquidities = pd.DataFrame.from_dict(liqs_dict)

In [143]:
liquidities['sum'] = liquidities.sum(axis=1)
liquidities = liquidities.sort_values('sum', ascending=False)
liquidities.head(20)

,trader_joe,pangolin,sum
avalanche-2,4.937020e+07,2.123553e+07,7.060573e+07
usd-coin,1.780000e+07,7.941046e+06,2.574104e+07
tether,7.472400e+06,3.303848e+06,1.077625e+07
joe,7.108440e+06,NaN,7.108440e+06
bitcoin-avalanche-bridged-btc-b,5.906218e+06,NaN,5.906218e+06
benqi-liquid-staked-avax,4.559416e+06,3.458475e+05,4.905264e+06
benqi,3.106253e+04,3.203974e+06,3.235037e+06
gmx,2.318141e+06,NaN,2.318141e+06
pangolin,NaN,2.239384e+06,2.239384e+06
wrapped-bitcoin,5.801959e+05,1.230443e+06,1.810638e+06


In [144]:
liq_check = (liquidities['sum'] > 2e6).sort_values(ascending=False)
liq_check_passed = liq_check[liq_check]

In [145]:
lq_passed = sp_passed[
    sp_passed.index.isin(liq_check_passed.index)
]

In [146]:
lq_passed

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
tether,Tether,USDT,6.853166e+10,3.0,1.001000,6.847311e+10,6.847311e+10,0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7
usd-coin,USD Coin,USDC,4.372039e+10,5.0,0.999666,4.383663e+10,4.375106e+10,0xb97ef9ef8734c71904d8002f8b6bc66dd9c48a6e
avalanche-2,Avalanche,AVAX,4.764716e+09,18.0,16.020000,4.122614e+08,2.975121e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
gmx,GMX,GMX,2.950153e+08,129.0,36.220000,8.148530e+06,8.148530e+06,0x62edc0692bd897d2295872a9ffcac5425011c661
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,7.767342e+07,303.0,16.750000,4.638167e+06,4.638167e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
bitcoin-avalanche-bridged-btc-b,Bitcoin Avalanche Bridged (BTC.b),BTCB,7.311927e+07,319.0,19542.390000,3.742069e+03,3.742069e+03,0x152b9d0fdc40c096757f570a51e494bd4b943e50
joe,JOE,JOE,6.861451e+07,334.0,0.200615,4.393236e+08,3.418220e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,2.548332e+07,602.0,0.008437,7.200000e+09,3.020811e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
pangolin,Pangolin,PNG,1.018275e+07,886.0,0.077153,2.300000e+08,1.320256e+08,0x60781c2586d68229fde47564546784ab3faca982


### Non quantative checks

In [147]:
exclude_list = [
    {
        'gecko_id': 'tether',
        'reason': 'stablecoin'
    },
    {
        'gecko_id': 'usd-coin',
        'reason': 'stablecoin'
    },
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Not a native Avalanche asset'
    },
]
exclude_ids = [token['gecko_id'] for token in exclude_list]

In [148]:
nq_passed = lq_passed[
    ~lq_passed.index.isin(exclude_ids)
]

In [149]:
nq_passed

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,4.764716e+09,18.0,16.020000,4.122614e+08,2.975121e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,7.767342e+07,303.0,16.750000,4.638167e+06,4.638167e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
bitcoin-avalanche-bridged-btc-b,Bitcoin Avalanche Bridged (BTC.b),BTCB,7.311927e+07,319.0,19542.390000,3.742069e+03,3.742069e+03,0x152b9d0fdc40c096757f570a51e494bd4b943e50
joe,JOE,JOE,6.861451e+07,334.0,0.200615,4.393236e+08,3.418220e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,2.548332e+07,602.0,0.008437,7.200000e+09,3.020811e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
pangolin,Pangolin,PNG,1.018275e+07,886.0,0.077153,2.300000e+08,1.320256e+08,0x60781c2586d68229fde47564546784ab3faca982


### Retrieve historical data & token age check

In [150]:
import time

In [151]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in nq_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < 6 * 30:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {6*30} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < 6 * 30:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {6 * 30} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding BTCB, prices data available only for 126 < 180 days


In [152]:
lt_passed = nq_passed[
    ~nq_passed.index.isin(exclude_list)
]

### Slippage check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [153]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e3
trade_value = 1e5

In [154]:
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

In [155]:
url_0x = "https://avalanche.api.0x.org/swap/v1/price"

In [156]:
decimals = 18
slippages = []
for id_, data in lt_passed.iterrows():
    query = {
        'buyToken': 'WETH',
        'sellToken': coins[id_]['addr'],
        'sellAmount': int(trade_value_tiny / data['current_price'] * 10 ** decimals),
    }
    
    # spot price is calculated as a price for 1000$ swap
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    spot_price = float(swap['price'])
    
    query['sellAmount'] = int(trade_value / data['current_price'] * 10 ** decimals)
    resp = requests.get(url_0x, params=query)
    swap = resp.json()
    del_price = float(swap['price'])
    
    slippage = del_price / spot_price - 1
    
    slippages.append({
        'symb': id_,
        'spot_price': spot_price,
        'delivery_price': del_price,
        'slippage': slippage
    })
    
    

In [157]:
slippages_df = pd.DataFrame(slippages)
slippages_df

,symb,spot_price,delivery_price,slippage
0,avalanche-2,0.011578,0.011543,-0.003074
1,benqi-liquid-staked-avax,0.012084,0.012045,-0.003227
2,joe,0.000145,0.000141,-0.033940
3,benqi,0.000006,0.000006,-0.061158
4,pangolin,0.000056,0.000051,-0.089056


In [158]:
exclude_list = list(slippages_df[abs(slippages_df['slippage']) > abs(max_slippage)]['symb'])
exclude_list

[]

In [159]:
st_passed = lt_passed[
    ~lt_passed.index.isin(exclude_list)
]

### Marketcap ranking & top10 filtering

In [160]:
chosen_tokens = st_passed.sort_values("market_cap", ascending=False).iloc[:10]

In [161]:
chosen_tokens

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,4.764716e+09,18.0,16.020000,4.122614e+08,2.975121e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,7.767342e+07,303.0,16.750000,4.638167e+06,4.638167e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,6.861451e+07,334.0,0.200615,4.393236e+08,3.418220e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,2.548332e+07,602.0,0.008437,7.200000e+09,3.020811e+09,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
pangolin,Pangolin,PNG,1.018275e+07,886.0,0.077153,2.300000e+08,1.320256e+08,0x60781c2586d68229fde47564546784ab3faca982


In [162]:
chosen_tokens_symbols = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_symbols]
marketcaps = marketcaps[chosen_tokens_symbols]

In [163]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [164]:
# exclude savax from weighting since it should be a part of avax weight
mcaps = marketcaps[marketcaps.columns.drop("benqi-liquid-staked-avax")]
weights = mcaps.div(mcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [165]:
weights

,avalanche-2,joe,benqi,pangolin
date,,,,
2022-03-04,0.984490,0.009696,0.004650,0.001164
2022-03-05,0.985521,0.009396,0.004023,0.001060
2022-03-06,0.985530,0.009331,0.004064,0.001076
2022-03-07,0.985217,0.009523,0.004175,0.001085
2022-03-08,0.985637,0.009230,0.004079,0.001054
...,...,...,...,...
2022-10-21,0.977720,0.014287,0.005372,0.002621
2022-10-22,0.978427,0.013996,0.005270,0.002306
2022-10-23,0.978157,0.014187,0.005440,0.002217


In [166]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [167]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [168]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [169]:
adjusted_weights = adjust_weights(weights, max_weight=0.5) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=0.005)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=0.5)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)
adjusted_weights

,avalanche-2,joe,benqi,pangolin
date,,,,
2022-03-04,0.5,0.312560,0.149914,0.037525
2022-03-05,0.5,0.324477,0.138917,0.036607
2022-03-06,0.5,0.322416,0.140411,0.037173
2022-03-07,0.5,0.322112,0.141197,0.036690
2022-03-08,0.5,0.321316,0.141987,0.036696
...,...,...,...,...
2022-10-21,0.5,0.320641,0.120550,0.058810
2022-10-22,0.5,0.324393,0.122153,0.053454
2022-10-23,0.5,0.324747,0.124514,0.050738


In [170]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
    range=[0,1],
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

### Split AVAX 

In [171]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    split = 0.2
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - split)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [172]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [173]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [174]:
converted_last_weights = convert_weights(last_weights)

## Summary table

In [175]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = marketcaps.iloc[-1].astype(int)
cai['price'] = prices_data.iloc[-1]
cai['weight'] = last_weights
cai['weight_converted'] = converted_last_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,4673816695,15.733817,0.400000,103,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,76432626,16.441767,0.100000,25,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be
joe,JOE,JOE,67752455,0.197920,0.328025,84,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi,BENQI,QI,25160783,0.008329,0.121816,31,0x8729438eb15e2c8b576fcc6aecda6a148776c0f5
pangolin,Pangolin,PNG,10360063,0.078467,0.050158,12,0x60781c2586d68229fde47564546784ab3faca982


In [176]:
print(cai[['address', 'weight_converted']].to_csv(None, index=False))

address,weight_converted
0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7,103
0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,25
0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,84
0x8729438eb15e2c8b576fcc6aecda6a148776c0f5,31
0x60781c2586d68229fde47564546784ab3faca982,12

